In [1]:
import pandas as pd
# Set display options
pd.set_option('display.max_columns', None)      # used to show all the data column
pd.set_option('display.max_colwidth', None)     # used to show the entire content
pd.set_option('display.width', None)            # expanding width for long data sentences.

In [2]:
df = pd.read_excel('/content/drive/MyDrive/new_data.xlsx')
df.head()

,Unnamed: 0,Original Text,Translation Text
0,0,I overheard him and his practices .,I overheard him and his plans .
1,1,"I have possessed your grace of what I purpose, And by our holy Sabbath have I sworn To have the due and forfeit of my bond.","Your grace, I have told you what my intention is, and I have sworn by the holy Sabbath to take what is owed to me by our legal contract."
2,2,"and against your cony-catching rascals, Bardolph, Nym, and Pistol.","and about your cheating scoundrels, Bardolph, Nym, and Pistol."
3,3,"But for your words , they rob the Hybla bees And leave them honeyless .","But your words are as sweet as honey , you've stolen from the bees and left them with nothing ."
4,4,O dear !,"Oh , dear !"


In [3]:
df = df.drop(columns=['Unnamed: 0'])

In [4]:
df.head()

,Original Text,Translation Text
0,I overheard him and his practices .,I overheard him and his plans .
1,"I have possessed your grace of what I purpose, And by our holy Sabbath have I sworn To have the due and forfeit of my bond.","Your grace, I have told you what my intention is, and I have sworn by the holy Sabbath to take what is owed to me by our legal contract."
2,"and against your cony-catching rascals, Bardolph, Nym, and Pistol.","and about your cheating scoundrels, Bardolph, Nym, and Pistol."
3,"But for your words , they rob the Hybla bees And leave them honeyless .","But your words are as sweet as honey , you've stolen from the bees and left them with nothing ."
4,O dear !,"Oh , dear !"


In [27]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 11.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers


In [ ]:
import random
from transformers import MarianMTModel, MarianTokenizer
# English to French translation model
model_name_en_fr = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer_en_fr = MarianTokenizer.from_pretrained(model_name_en_fr)
model_en_fr = MarianMTModel.from_pretrained(model_name_en_fr)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# French to English translation model
model_name_fr_en = 'Helsinki-NLP/opus-mt-fr-en'
tokenizer_fr_en = MarianTokenizer.from_pretrained(model_name_fr_en)
model_fr_en = MarianMTModel.from_pretrained(model_name_fr_en)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def translate(text, tokenizer, model, max_length=512):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=max_length, truncation=True)
    translated_tokens = model.generate(inputs, max_length=max_length, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text


In [ ]:
from tqdm import tqdm
batch_size = 500
back_translated_texts = []

for batch_num, start_idx in enumerate(tqdm(range(0, len(sample_df), batch_size), desc="Translating batches")):
    end_idx = min(start_idx + batch_size, len(sample_df))
    batch_texts = sample_df['Modern English'][start_idx:end_idx]
    batch_back_translated = []

    for text in batch_texts:
        try:
            # Translate to French
            french_text = translate(text, tokenizer_en_fr, model_en_fr)
            # Translate back to English
            back_translated_text = translate(french_text, tokenizer_fr_en, model_fr_en)
            batch_back_translated.append(back_translated_text)
        except Exception as e:
            print(f"Error while translating index {start_idx}: {e}")
            batch_back_translated.append(text)  # Keep the original if translation fails

    back_translated_texts.extend(batch_back_translated)
    print(f"Completed batch {batch_num + 1} out of {len(range(0, len(sample_df), batch_size))}")


Translating batches:   2%|▎         | 1/40 [06:32<4:14:56, 392.21s/it]

Completed batch 1 out of 40


Translating batches:   5%|▌         | 2/40 [12:54<4:04:42, 386.38s/it]

Completed batch 2 out of 40


Translating batches:   8%|▊         | 3/40 [19:03<3:53:20, 378.39s/it]

...

Completed batch 38 out of 40


Translating batches:  98%|█████████▊| 39/40 [4:01:36<06:15, 375.85s/it]

Completed batch 39 out of 40


Translating batches: 100%|██████████| 40/40 [4:07:39<00:00, 371.49s/it]

Completed batch 40 out of 40


In [ ]:
sample_df['Back_Translated'] = back_translated_texts

In [ ]:
sample_df.to_excel('/content/drive/MyDrive/augmented_dataset.xlsx', index=False)

In [31]:
augmented_df= pd.read_excel('/content/drive/MyDrive/augmented_dataset.xlsx')


In [32]:
augmented_df.head()

,Shakespearen English,Modern English,Back_Translated
0,"i'll after him , and see the event of this .",i will go too and make sure that happens .,I'll also make sure it happens.
1,di faciant laudis summa sit ista tuae!,may the gods ensure that this is the high point of your glory.,Let the gods make sure that this is the culmination of your glory.
2,"thou takest up thou know'st not what; but take it for thy labour: it is a thing I made, which hath the king five times redeem'd from death: I do not know what is more cordial.","you do not know what you are picking up. but take it in exchange for your work. it is something I made, which saved the king from death five different times. I do not know of anything that is better for your health.","You don't know what you're collecting, but take it in exchange for your work. It's something I did, which saved the king from death five times different. I don't know anything better for your health."
3,"my stay to you a charge and trouble: to save both, farewell, our brother.","if I stay, i will only be a burden to your hospitality. so if I leave, it is a win-win! goodbye, brother.","If I stay, I'll only be a burden for your hospitality, so if I leave, it's a win-win!"
4,"and as they last, their verdure still endure, to drive infection from the dangerous year!","as long as your fresh, young lips are around, the world will be free from disease all year round!","As long as your fresh and young lips are around, the world will be free from disease all year round!"


In [35]:
identical_rows = augmented_df[augmented_df['Modern English'] == augmented_df['Back_Translated']]
number_of_identical_rows = identical_rows.shape[0]
print(f"Number of identical rows: {number_of_identical_rows}")

Number of identical rows: 160


In [36]:
augmented_df = augmented_df.drop(identical_rows.index)
augmented_df = augmented_df.reset_index(drop=True)

In [39]:
identical_rows = augmented_df[augmented_df['Modern English'] == augmented_df['Back_Translated']]
number_of_identical_rows = identical_rows.shape[0]
print(f"Number of identical rows: {number_of_identical_rows}")

Number of identical rows: 0


In [38]:
number_of_empty_values = augmented_df['Back_Translated'].isna().sum()
print(f"Number of empty values in 'Back_Translated' column: {number_of_empty_values}")

Number of empty values in 'Back_Translated' column: 0


In [40]:
augmented_df = augmented_df.drop(columns=['Modern English'])
augmented_df = augmented_df.rename(columns={'Back_Translated': 'Modern English'})

In [41]:
augmented_df.head()

,Shakespearen English,Modern English
0,"i'll after him , and see the event of this .",I'll also make sure it happens.
1,di faciant laudis summa sit ista tuae!,Let the gods make sure that this is the culmination of your glory.
2,"thou takest up thou know'st not what; but take it for thy labour: it is a thing I made, which hath the king five times redeem'd from death: I do not know what is more cordial.","You don't know what you're collecting, but take it in exchange for your work. It's something I did, which saved the king from death five times different. I don't know anything better for your health."
3,"my stay to you a charge and trouble: to save both, farewell, our brother.","If I stay, I'll only be a burden for your hospitality, so if I leave, it's a win-win!"
4,"and as they last, their verdure still endure, to drive infection from the dangerous year!","As long as your fresh and young lips are around, the world will be free from disease all year round!"


In [42]:
df.shape

(70236, 2)

In [44]:
df = pd.concat([df, augmented_df], ignore_index=True)

In [45]:
df.shape

(90076, 2)

In [46]:
df.to_excel('/content/drive/MyDrive/NLPStyleTransfer_dataset.xlsx', index=False)